# 4. Processing of the "Construction of non residentials" dataset

## almost identical to ```processing_constr_resid```

### Importing libraries and dataset

In [ ]:
import pandas as pd
import re
from matplotlib import pyplot as plt 
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
# pip install "folium>=0.12" matplotlib mapclassify
# pip install xlrd

# pip install keplergl


In [ ]:
df = pd.read_csv("https://minio.lab.sspcloud.fr/mligeret1/constructions_non_resid_geocoded.csv",sep=",",on_bad_lines="warn")


In [ ]:
df.head()
df = df.iloc[1:]
df.head()
df[df["result_status"] == "error"].loc[:,"Adresse_complete"]


### Grouping columns together
Pour se faire, on regroupe les colonnes de même ordre  


In [ ]:
for c in df.columns:
    print(c)

In [ ]:
col_metadata = [x for x in df.columns if "DAU" in x] + ["Unnamed: 0"]
col_meta_location = [x for x in df.columns if "lieu des travaux" in x]
col_dates_travaux = [x for x in df.columns if "Date" in x]
col_demandeurs = [x for x in df.columns if "demandeur" in x and ("Nature du projet déclarée par le demandeur" not in x)]
col_precise_location = [x for x in df.columns if "du terrain" in x and ("Superficie du terrain" not in x)] 
col_cadastres = [x for x in df.columns if "cadastre" in x]
col_construction_details = [x for x in df.columns if "Indicateur" in x] + ["Nombre de niveaux du bâtiment le plus élevé","Nombre de chambres (hôtels et autres hébergements, notamment sociaux)"]
col_details_transf = [x for x in df.columns if "Surface de plancher" in x]
col_from_geocodage = [i for i in df.columns if "result_" in i]

col_irrelevant = col_metadata + col_dates_travaux + col_precise_location + col_cadastres + col_construction_details + col_details_transf + col_from_geocodage
col_relevant =  [c for c in df.columns if c not in col_irrelevant] 
for c in col_relevant:
    print(c)

### Arranging columns types 


In [ ]:
dico_variables = pd.read_excel("https://minio.lab.sspcloud.fr/mligeret1/dictionnaire-variables-locaux-permis-construire-20220531.xls")

In [ ]:
dico_variables.head()


#### Retrouver depuis le dictionnaire 

In [ ]:
def variable_types(ligne):
    if ("Année" in ligne["Description de la variable"]):
        return None
    if "Alphanumérique" in ligne["Format"]:
        return "string"
    if "Numérique" in ligne["Format"]:
        return "Int64"
        
dico_variables["Format_python"] = dico_variables.apply(variable_types, axis=1)


In [ ]:
dtype_map = dict(zip(dico_variables["Description de la variable"], dico_variables["Format_python"]))
dtype_map = {col : python_type for col, python_type in dtype_map.items() if col in df.columns}
df = df.astype(dtype_map, errors="ignore")

#### Régler la question de la date 

In [ ]:
df["Date (mois) de prise en compte (DPC) du premier évènement reçu dans Sitadel (dépôt de la demande ou autorisation)"].value_counts().to_frame().style #to verify that dates are well specified under the format %Y%m, i.e. year with 4 digits followed by month
df["Date (mois) de prise en compte (DPC) du premier évènement reçu dans Sitadel (dépôt de la demande ou autorisation)"] = pd.to_datetime(df["Date (mois) de prise en compte (DPC) du premier évènement reçu dans Sitadel (dépôt de la demande ou autorisation)"], format="%Y-%m", errors="coerce")


We repeat the operation for the other columns that involve date/year dtypes


In [ ]:
for col_date in col_dates_travaux:
    print(df[col_date].value_counts().to_frame()) #to verify the format of the date

In [ ]:
col_date_format_ymd = [col_dates_travaux[i] for i in [0,1,2,3]]
col_date_format_ym = [col_dates_travaux[i] for i in [4,5,6]]

In [ ]:
for col in col_date_format_ymd:
    print(df[col])
    df[col] = pd.to_datetime(df[col], format="%Y-%m-%d")

In [ ]:
for col in col_date_format_ym:
    print(df[col])
    df[col] = pd.to_datetime(df[col], format="%Y-%m")


In [ ]:
col = "Année de dépôt de la DAU"
print(df[col])
df[col] = df[col].astype("int64")

#### Compléter les dernières colonnes

In [ ]:
df = df.astype({col : "float64" for col in col_details_transf + ["Superficie du terrain"]})

df = df.astype({col : "string" for col in col_meta_location + col_cadastres + ["Code zone opératoire"]})
df = df.astype({col : "string" for col in ["Adresse_complete"]})

In [ ]:
df.dtypes.to_frame().style

### Cleaning per column



#### Cleaning dates 


In [ ]:
df.loc[:,col_dates_travaux].sample(20)

On regarde si les dates mis à part la colonne des années sont cohérentes, ce qui semble être le cas 

In [ ]:
df[df[col_dates_travaux].gt(pd.Timestamp.today()).any(axis=1)][col_dates_travaux+["Année de dépôt de la DAU"]].head(20)


In [ ]:
df[df["Année de dépôt de la DAU"]>2025][col_dates_travaux+["Année de dépôt de la DAU"]].head(20)


In [ ]:
ligns_gt_today = df["Année de dépôt de la DAU"]>2025
df.loc[ligns_gt_today,"Année de dépôt de la DAU"] = df.loc[ligns_gt_today,"Date (mois) de prise en compte (DPC) du premier évènement reçu dans Sitadel (dépôt de la demande ou autorisation)"].dt.year

In [ ]:
df[df["Année de dépôt de la DAU"]>2025][col_dates_travaux+["Année de dépôt de la DAU"]].head(20)


On cherche à voir les années avant lesquelles il y a peu de données et qu'il vaut mieux supprimer pour l'analyse statistique 

In [ ]:
df["Année de dépôt de la DAU"].value_counts().sort_index().plot.bar()


On peut supprimer tout ce qui vient avant 2012, 2012 compris ainsi que l'année 2025 qui est en cours 

In [ ]:
ligns_few_values = (df["Année de dépôt de la DAU"]<=2012) | (df["Année de dépôt de la DAU"] == 2025)
df = df.drop(df.index[ligns_few_values])

In [ ]:
df["Année de dépôt de la DAU"].value_counts().sort_index().plot.bar()


#### Cleaning Departments 

In [ ]:
df["Code du département du lieu des travaux - Code de la zone"].nunique()

On remarque que le département 2 et 02 est codé deux fois, on vérifie que c'est bien le même département, puis on renomme 2 en 02, de même pour les autres départements, cela permettra d'avoir des séries bien ordonnées dans l'ordre

In [ ]:
df.loc[(df["Code du département du lieu des travaux - Code de la zone"]=="2") | (df["Code du département du lieu des travaux - Code de la zone"]=="02"), col_meta_location].sample(10)

In [ ]:
df.loc[df["Code du département du lieu des travaux - Code de la zone"].str.len()==1,"Code du département du lieu des travaux - Code de la zone"] 

In [ ]:
df["Code du département du lieu des travaux - Code de la zone"]=df["Code du département du lieu des travaux - Code de la zone"].str.zfill(2)

#### Isolating overseas departments (DOM)

In [ ]:
df_dom = df.loc[df["Code du département du lieu des travaux - Code de la zone"].str.startswith("97")] 
df = df.drop(df_dom.index)
df_dom.groupby(["Code du département du lieu des travaux - Code de la zone", "Année de dépôt de la DAU"]).size().unstack(0).sort_index().plot()

#### Replacing qualitative numeric value by its name 

In [ ]:
destinations = [
    "habitation",
    "hébergement hôtelier",
    "bureaux",
    "commerce",
    "artisanat",
    "industrie",
    "agriculture",
    "entrepôt",
    "service public ou d'intérêt collectif"
]

dict_destination_principale = {key: value for key, value in zip(range(1, 10), destinations)}


In [ ]:
df.loc[~df["Destination principale"].isin(range(1, 10)),"Destination principale"]

In [ ]:

df["Destination principale"] = pd.to_numeric(df["Destination principale"],errors="coerce")
df["Destination principale"] = df["Destination principale"].apply(lambda x : dict_destination_principale[x])


# 

## Descriptive statistics 

### Main usage of the building before and after 

In [ ]:
df["Destination principale"].value_counts()

In [ ]:
df["Type principal des locaux d'origine transformés"].value_counts()

### Construction Activity Time Series

In [ ]:
df["Année de dépôt de la DAU"].value_counts().sort_index().plot()


In [ ]:
df["Date"] = df["Date (mois) de prise en compte (DPC) du premier évènement reçu dans Sitadel (dépôt de la demande ou autorisation)"]
df[df["Date"].dt.year<=2012][col_dates_travaux+["Année de dépôt de la DAU"]].head()


### Question : In which departments did we build the most ? 

In [ ]:
activity_per_department = df.groupby(["Code du département du lieu des travaux - Code de la zone", "Année de dépôt de la DAU"]).size().unstack(0).sort_index().plot(legend=False)

In [ ]:
df.groupby(["Code du département du lieu des travaux - Code de la zone", "Année de dépôt de la DAU"]).size().unstack(0).sort_index().plot(
    subplots=True,
    layout=(10,10),
    figsize=(15,15),
    legend=True 
)


In [ ]:
activity_per_department = df.groupby(["Code du département du lieu des travaux - Code de la zone", "Année de dépôt de la DAU"]).size().unstack(1)
activity_per_department.head(50)

It's not very convenient to read and analyze, let's try making it clearer 

Let's first determine for a given year, in which departments did we build the most 
#### Q1 : Let's focus on the year 2018

In [ ]:
construction_activity2018 = activity_per_department[2018].sort_values().reset_index()

In [ ]:
construction_activity2018.nunique()

#### -> Q1 : There are some departments where nothing was built ... 

In [ ]:
construction_activity2018["Code du département du lieu des travaux - Code de la zone"].nunique()

In [ ]:
construction_activity2018.loc[construction_activity2018[2018]==None]

#### -> Q1 : Let's represent departments on a map 

In [ ]:
deps = gpd.read_file("https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements.geojson")

In [ ]:
deps.index = deps.index.astype(str).str.zfill(2)
deps.head(50)


In [ ]:
construction_activity2018 = construction_activity2018.merge(
    deps,
    left_on="Code du département du lieu des travaux - Code de la zone",
    right_on="code",
    how="left"
)

construction_activity2018.head(50)


In [ ]:
construction_activity2018 = gpd.GeoDataFrame(
    construction_activity2018,
    geometry="geometry",
    crs=deps.crs
)

In [ ]:
construction_activity2018.plot(
    column=2018,
    cmap="OrRd",
    legend=True,
    figsize=(10, 10),
    edgecolor="black"
) 

#### -> Q1 What's happening in the North ? 

In [ ]:
df.loc[
    df["Code du département du lieu des travaux - Code de la zone"] == "44",
    "Destination principale"
].value_counts()


On peut supposer qu'il y a plusieurs biais dans la base, étant donné que les administrations sont plus susceptibles de signaler des travaux que des acteurs privés, de la même manière l'agriculture code plus fréquemment des modifications de terrains 

#### Q2 : In which department was growth the strongest ? 

In [ ]:
growth_per_department = activity_per_department.pct_change(axis=1)



In [ ]:
growth_per_department = growth_per_department.mean(axis=1).reset_index()


In [ ]:
growth_per_department = growth_per_department.merge(
    deps,
    left_on="Code du département du lieu des travaux - Code de la zone",
    right_on="code",
    how="left"
)

In [ ]:
growth_per_department = growth_per_department.rename(columns={0:"Avg_growth_rate"})

In [ ]:
growth_per_department = gpd.GeoDataFrame(
    growth_per_department,
    geometry="geometry",
    crs=deps.crs
)

In [ ]:
growth_per_department.columns

In [ ]:
growth_per_department.plot(
    column="Avg_growth_rate",
    cmap="OrRd",
    legend=True,
    figsize=(10, 10),
    edgecolor="black"
) 

In [ ]:
growth_per_department.sort_values(by="Avg_growth_rate", ascending=False).head(10)

#### -> Q2 : Can this be explained by population growth ?


In [ ]:
deps_pop = pd.read_excel("https://www.insee.fr/fr/statistiques/fichier/2012713/TCRD_004.xlsx")

In [ ]:

deps_pop.columns = deps_pop.iloc[2]



In [ ]:

deps_pop = deps_pop.iloc[3:]

deps_pop.head()


In [ ]:
deps_pop = deps_pop.reset_index()


In [ ]:
deps_pop.head()

In [ ]:
deps_pop.columns = ["to_del", "deps_code", "deps_name", "2025", "share_pop","2022", "2016","2011","1999"]


In [ ]:
deps_pop = deps_pop.loc[0:101,["deps_code","2011","2025"]]

In [ ]:
deps_pop.set_index("deps_code")


In [ ]:
deps_pop = deps_pop.loc[:100,]


In [ ]:
deps_pop[["2011", "2025"]] = deps_pop[["2011", "2025"]].astype(float, errors="ignore")


In [ ]:
deps_pop


In [ ]:
deps_pop["pop_growth_2011-2025"] = deps_pop[["2011","2025"]].pct_change(axis=1)["2025"]


In [ ]:
deps_pop_growth = deps_pop.loc[:,["deps_code","pop_growth_2011-2025"]]


In [ ]:
growth_per_department = growth_per_department.merge(
    deps_pop_growth,
    left_on="Code du département du lieu des travaux - Code de la zone",
    right_on="deps_code",
    how="left"
)

In [ ]:
growth_per_department
plt.scatter(
    growth_per_department["Avg_growth_rate"],
    growth_per_department["pop_growth_2011-2025"],
)
growth_per_department[["Avg_growth_rate", "pop_growth_2011-2025"]].corr()


The coefficient is rather low 